In [1]:
import h5py
import numpy as np
import xarray as xr
import datetime
import pandas as pd
from salishsea_tools import utilities
import os

In [2]:
f = h5py.File('temp/St_georgia.hdf5', 'r')

In [91]:
check = False
grid_z = np.arange(40)
grid_y = np.arange(396)
grid_x = np.arange(896)
bar = utilities.statusbar('Loading ...')
for i in bar(range(168)):
    i = i + 1
    stamp = '0' * (5-len(str(i))) + str(i)
    time = np.asarray(f['Time'][f'Time_{stamp}']).astype(int)
    time = datetime.datetime(time[0], time[1], time[2], time[3], time[4], time[5])
    currentu = np.expand_dims(np.asarray(f['Results']['velocity U'][f'velocity U_{stamp}']).astype('float32'),0)
    currentv = np.expand_dims(np.asarray(f['Results']['velocity V'][f'velocity V_{stamp}']).astype('float32'),0)
    water = np.expand_dims(np.asarray(f['Results']['water level'][f'water level_{stamp}']).astype('float32'),0)
    u = xr.DataArray(currentu, coords  = [[time], grid_z, grid_y, grid_x], dims= ['time', 'depth', 'grid_y', 'grid_x'])
    v = xr.DataArray(currentv, coords  = [[time], grid_z, grid_y, grid_x], dims= ['time', 'depth', 'grid_y', 'grid_x'])
    w = xr.DataArray(water, coords  = [[time], grid_y, grid_x], dims= ['time', 'grid_y', 'grid_x'])
    a = xr.Dataset({'current_u': u, 'current_v':v, 'ssh' : w})
    if check == False: 
        check = True
        chunk = a
    else:
        chunk = xr.concat((chunk, a), dim = 'time')
    piece = i / 12
    if piece == int(piece):
        check = False
        comp  = {'time':{'zlib':True, 'complevel': 4},
                 'grid_x':{'zlib':True, 'complevel': 4, 'dtype':'float32'},
                 'grid_y':{'zlib':True, 'complevel': 4, 'dtype':'float32'},
                 'depth':{'zlib':True, 'complevel': 4, 'dtype':'float32'}}
        filenm = f'temp/{str(int(piece))}.nc'
        tempname = f'temp/temp{i}.nc'
        chunk.to_netcdf(f'temp/temp{i}.nc', format = 'NETCDF4',engine = 'netcdf4', encoding = comp)
        os.system(f"nccopy -d4 {tempname} {filenm}") 

Loading ... 100% (168 of 168) |#################################|Time:  0:15:53


In [2]:
listr = []
for i  in range(14):
    listr.append(xr.open_dataset(f'temp/{i+1}.nc'))

In [4]:
concat = xr.concat(listr, dim = 'time')

In [ ]:
concat.to_netcdf(f'temp/current_ssh.nc',format = 'NETCDF4',engine = 'netcdf4')

In [ ]:
os.system('shutdown -s')